In [52]:
import sys
!{sys.executable} -m pip install regex

#upyter contrib nbextension install --user


In [60]:
import spacy
from nltk import Tree
from spacy import displacy
import copy

import regex # it replace re I think

# i dont remind. I think there is no need to install, just download last realese and set the next path
nlp = spacy.load('/home/jason/Downloads/en_core_web_sm-2.0.0/en_core_web_sm/en_core_web_sm-2.0.0')

In [2]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

doc = nlp(u"a temporary state gasoline tax to raise money for earthquake relief.")
[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

                      tax                            
  _____________________|_______________               
 |      |       |      |      |      raise           
 |      |       |      |      |    ____|_______       
 |      |       |      |      |   |    |      for    
 |      |       |      |      |   |    |       |      
 |      |       |      |      |   |    |     relief  
 |      |       |      |      |   |    |       |      
 a  temporary state gasoline  .   to money earthquake



[None]

In [29]:
doc = "Westinghouse Electric Corp."
doc = unicode(text, 'utf-8')
doc = nlp(text)
display_dep(doc, False)

Corp. 22 27


In [111]:
def display_dep(doc, merge = True, entities=False):
    doc2 = copy.deepcopy(doc) # deep (recursive) copy
    if merge:
        for np in doc2.noun_chunks:
            # By default the noun phrase will be now be labeled the same as the root node
            np.merge()
    
    displacy.render(doc2, style='dep',jupyter=True)
    if entities:
        displacy.render(doc, style='ent',jupyter=True)
def traverse_tree():
    for token in example_doc:    
        print ([token.i, token.text, token.dep_, token.head.text,token.head.pos_,[child for child in token.children]])

In [27]:
for x in dir(h):
    print (x, getattr(h,x))

('_', <spacy.tokens.underscore.Underscore object at 0x7f3845af01d0>)
('__bytes__', <built-in method __bytes__ of spacy.tokens.token.Token object at 0x7f3845202680>)
('__class__', <type 'spacy.tokens.token.Token'>)
('__delattr__', <method-wrapper '__delattr__' of spacy.tokens.token.Token object at 0x7f3845202680>)
('__doc__', 'An individual token \xe2\x80\x93 i.e. a word, punctuation symbol, whitespace,\n    etc.')
('__eq__', <method-wrapper '__eq__' of spacy.tokens.token.Token object at 0x7f3845202680>)
('__format__', <built-in method __format__ of spacy.tokens.token.Token object at 0x7f3845202680>)
('__ge__', <method-wrapper '__ge__' of spacy.tokens.token.Token object at 0x7f3845202680>)
('__getattribute__', <method-wrapper '__getattribute__' of spacy.tokens.token.Token object at 0x7f3845202680>)
('__gt__', <method-wrapper '__gt__' of spacy.tokens.token.Token object at 0x7f3845202680>)
('__hash__', <method-wrapper '__hash__' of spacy.tokens.token.Token object at 0x7f3845202680>)
('__i

In [116]:
adj_doc = "The company's customer service was terrible."
verb_doc = "They kept increasing my phone bill"
#example_doc = "The compound dependency identifies compound nouns."
example_doc = "its fight against the government and the policies of racial segregation on all fronts including the armed struggle ."

adj_doc = unicode(adj_doc, 'utf-8')
verb_doc = unicode(verb_doc, 'utf-8')
example_doc = unicode(example_doc, 'utf-8')

adj_doc = nlp(adj_doc)
verb_doc = nlp(verb_doc)
example_doc = nlp(example_doc)

def get_compound_pairs(doc, verbose=False):
    """Return tuples of (multi-noun word, adjective or verb) for document."""
    compounds = [tok for tok in doc if tok.dep_ == 'compound'] # Get list of compounds in doc
    compounds = [c for c in compounds if c.i == 0 or doc[c.i - 1].dep_ != 'compound'] # Remove middle parts of compound nouns, but avoid index errors
    tuple_list = []
    if compounds: 
        for tok in compounds:
            pair_item_1, pair_item_2 = (False, False) # initialize false variables
            noun = doc[tok.i: tok.head.i + 1]
            pair_item_1 = noun
            # If noun is in the subject, we may be looking for adjective in predicate
            # In simple cases, this would mean that the noun shares a head with the adjective
            if noun.root.dep_ == 'nsubj':
                adj_list = [r for r in noun.root.head.rights if r.pos_ == 'ADJ']
                if adj_list:
                    pair_item_2 = adj_list[0] 
                if verbose == True: # For trying different dependency tree parsing rules
                    print("Noun: ", noun)
                    print("Noun root: ", noun.root)
                    print("Noun root head: ", noun.root.head)
                    print("Noun root head rights: ", [r for r in noun.root.head.rights if r.pos_ == 'ADJ'])
            if noun.root.dep_ == 'dobj':
                verb_ancestor_list = [a for a in noun.root.ancestors if a.pos_ == 'VERB']
                if verb_ancestor_list:
                    pair_item_2 = verb_ancestor_list[0]
                if verbose == True: # For trying different dependency tree parsing rules
                    print("Noun: ", noun)
                    print("Noun root: ", noun.root)
                    print("Noun root head: ", noun.root.head)
                    print("Noun root head verb ancestors: ", [a for a in noun.root.ancestors if a.pos_ == 'VERB'])
            if pair_item_1 and pair_item_2:
                tuple_list.append((pair_item_1, pair_item_2))
    return tuple_list


print get_compound_pairs(adj_doc)
print get_compound_pairs(verb_doc)
print get_compound_pairs(example_doc)
print get_compound_pairs(doc)

def get_head_info(doc, char_index = False):
    sentence_head = doc.sents.next().root
    head = sentence_head.text
    head_pos = sentence_head.pos_
    if char_index:
        off_head0 = sentence_head.idx
        off_head1 = off_head0 + len(sentence_head)
    else: 
        # token_index
        off_head0 = sentence_head.i
        off_head1 = off_head0
    print head, head_pos, off_head0, off_head1
    return head, head_pos, off_head0, off_head1

def get_compounds(doc):
    # list of compounds in the form [modifier#head] (or [compound, compound.head]) 
    compounds = []
    for tok in doc:
        if tok.dep_ == 'compound':
            tok_head = tok.head.text
            tok = tok.text        
            compounds.append(tok+"#"+tok_head)
    print (compounds)
    return compounds

[(customer service, terrible)]
[(phone bill, increasing)]
[]
[(syrup form, develop)]


In [117]:
#display_dep(verb_doc,False)
#display_dep(adj_doc,False)
display_dep(example_doc,False)

In [16]:
attrs = dir(h)
# {'kids': 0, 'name': 'Dog', 'color': 'Spotted', 'age': 10, 'legs': 2, 'smell': 'Alot'}
# now dump this in some way or another
print ', '.join("%s: %s" % item for item in attrs.items())

AttributeError: 'list' object has no attribute 'items'

In [16]:
l1 = [408,
411,
411,
412,
412,
413,
414]

l2 = [408,
412,
413,
412,
413,
413,
414]

included = []
output = []

for i, l in enumerate(l1):
    new_start  = l
    new_end  = l2[i]
    for j, tup in enumerate(included):
        start = tup[0]
        end = tup[1]
        is_already = (start <= new_start  and  new_end <= end)
        if is_already:
            output.append(0)
            break
    if not is_already:
        included.append((new_start,new_end))
        output.append(1)
print output, included

[1, 1, 1, 0, 0, 0, 1] [(408, 408), (411, 412), (411, 413), (414, 414)]


In [100]:
example_doc = "critics have charged AZT's maker Burroughs Wellcome Co. with corporate inertia because children account for just 1 % of the patient population and hence a small part of the large and lucrative market . Wellcome has replied that it is moving ahead to compile the relevant data and recently promised to develop a pediatric syrup form easier for youngsters to take . Still"
example_doc = unicode(example_doc, 'utf-8')
doc = nlp(example_doc)
print example_doc
for token in doc:
    print token, "\t\t", token.pos_

#"""
print "-----------------------"
doc2 = copy.deepcopy(doc) # deep (recursive) copy
for np in doc2.noun_chunks:
    # By default the noun phrase will be now be labeled the same as the root node
    np.merge(np.root.tag_, np.root.lemma_, np.root.ent_type_)    
for token in doc2:
    print token, "\t\t\t", token.pos_
#"""

critics have charged AZT's maker Burroughs Wellcome Co. with corporate inertia because children account for just 1 % of the patient population and hence a small part of the large and lucrative market . Wellcome has replied that it is moving ahead to compile the relevant data and recently promised to develop a pediatric syrup form easier for youngsters to take . Still
critics 		NOUN
have 		VERB
charged 		VERB
AZT 		PROPN
's 		PART
maker 		NOUN
Burroughs 		PROPN
Wellcome 		PROPN
Co. 		PROPN
with 		ADP
corporate 		ADJ
inertia 		NOUN
because 		ADP
children 		NOUN
account 		VERB
for 		ADP
just 		ADV
1 		NUM
% 		NOUN
of 		ADP
the 		DET
patient 		ADJ
population 		NOUN
and 		CCONJ
hence 		ADV
a 		DET
small 		ADJ
part 		NOUN
of 		ADP
the 		DET
large 		ADJ
and 		CCONJ
lucrative 		ADJ
market 		NOUN
. 		PUNCT
Wellcome 		PROPN
has 		VERB
replied 		VERB
that 		ADP
it 		PRON
is 		VERB
moving 		VERB
ahead 		ADV
to 		PART
compile 		VERB
the 		DET
relevant 		ADJ
data 		NOUN
and 		CCONJ
recently 		ADV
pr

In [88]:
def extract_with_POS(doc, pattern, merge=False):
    # merge makes it slow, but it could cover many noun_phrase variations
    # r' means it will ignore escapes. I dont know in which cases it is useful for POS
    # e.g. '\n' is treated as a newline character, while r'\n' is treated as the characters \ followed by n
    #pattern = r'(<ADJ>)*(<NOUN>)+(<ADP>)+(<ADJ>)*(<NOUN>)+' 

    doc2 = copy.deepcopy(doc) # deep (recursive) copy
    if merge:
        for np in doc2.noun_chunks:
            # By default the noun phrase will be now be labeled the same as the root node
            np.merge(np.root.tag_, np.root.lemma_, np.root.ent_type_) 
    
    ## create a string with the pos of the sentence
    posString = ""
    for w in doc2:
        posString += "<" + w.pos_ + ">"

    matched_list = []
    #for m in re.compile(pattern).finditer(posString):
    #  regex.BESTMATCH maybe its inclusion can helps
    for m in regex.finditer(pattern, posString, overlapped=True):
        # each m is a matched pattern. Now we need token-based offsets
        
        # count the "<" to find the token-based index (or the number of tokens that came before that group)
        # first we need to convert char-based offset (from re) into token-based offset (from spacy)
        group_token_index = posString[:m.start()].count('<') 
        
        # count the "<" in m tell us the number of tokens to extract
        numTokensInGroup = m.group().count('<')

        extracted_substring = doc2[group_token_index:group_token_index+numTokensInGroup]
        
        ## starting at character offset m.start()
        matched_list.append(extracted_substring)

    return matched_list


In [108]:
# print extract_with_POS(doc, "(<ADJ>)*(<NOUN>)+(<ADP>)+(<DET>)*(<ADJ>)*(<NOUN>)+") #version with merge =false 
# add proper noun
nouns_1="(<NOUN>|<PROPN>)+"
#nouns_1="(<NOUN>)+"
nouns_2="(<NOUN>|<PROPN>){2,}"
prep_phrase = extract_with_POS(doc, nouns_1+"(<ADP>)+"+nouns_1,merge=True)
compound_nouns = extract_with_POS(doc, "(<ADJ>)"+nouns_2, merge=False)
apostrophe = extract_with_POS(doc, nouns_1+"(<PART>)+"+nouns_1)

print doc
print "------ PREP"
print prep_phrase
print "------ COMPOUND"
print compound_nouns
print "------ APOSTROPHE"
print apostrophe

critics have charged AZT's maker Burroughs Wellcome Co. with corporate inertia because children account for just 1 % of the patient population and hence a small part of the large and lucrative market . Wellcome has replied that it is moving ahead to compile the relevant data and recently promised to develop a pediatric syrup form easier for youngsters to take . Still
------ PREP
[AZT's maker Burroughs Wellcome Co. with corporate inertia, Burroughs Wellcome Co. with corporate inertia, corporate inertia because children, just 1 % of the patient population, hence a small part of the large and lucrative market]
------ COMPOUND
[pediatric syrup form]
------ APOSTROPHE
[AZT's maker Burroughs Wellcome Co.]


In [94]:
# bestmatch usage
# (?:foo){s<=2} // matches "foo", permitting 2 substitutions.
print regex.findall(r'(?:foo){s<=2}', 'xxfoo')
print regex.findall(r'(?:foo){s<=2}', 'xxfoo', regex.BESTMATCH)

['xfo']
['foo']


[no machinery for a plant, a plant on that scale]
[its farm experts, farm experts]


In [150]:
def is_custom_stopword(pos):    
    if pos[0] == "J": # is adjetive
        return False
    elif pos[0:2] == "NN": # is noun
        return False
    elif pos[0:2] == "RB": # is adverb
        return False    
    elif pos[0:2] == "VB": # is verb
        return False
    elif pos[0:2] == "XX": # is unknown
        return False    
    else: return True

# POS
example_doc = "critics have had charged AZT's maker Burroughs Wellcome can be one of Co. with corporate inertia because children account for just 1 % of the patient population and hence a small part of the large and lucrative market . Wellcome has replied that it is moving ahead to compile the relevant data and recently promised to develop a pediatric syrup form easier for youngsters to take . Still"
example_doc = unicode(example_doc, 'utf-8')
doc = nlp(example_doc)
lexical=[]
for token in doc: 
    if not is_custom_stopword(token.tag_):
       lexical.append(token.string())
       #print token, "\t\t", token.tag_
print lexical

TypeError: 'unicode' object is not callable

In [160]:
mylongstring = "`` The Famous Teddy Z , '' which CBS Inc. had hoped would emerge as one of the few bright spots in its otherwise lackluster prime - time schedule centers on a mailroom clerk turned agent at Hollywood talent agency half hour situation comedy seen Mondays 9:30 p.m . Eastern and Pacific program series show it It network We we us coveted p.m. slot lead character 's beginning premiere premiered Greek family that Craig Nelson story consultant Mr. viewers who have not been with since Those Murphy Brown about television news magazine starring Candice Bergen estimated 2 % fall imports accompanied by growing tendency for Japanese manufacturers to build vehicles overseas slip exports Japan Automobile Manufacturers ' Association association Domestic demand spokesman an He new tax though source general resentment among taxpayers 3 consumption Robert J. Penn president chief executive officer 58 years old reorganized company this steelmaker UNR William S. Leavitt chairman 37 His father David his board 16 nation alliance NATO ministers concluding two day meeting southern Portugal defense court order allowing former National Security Adviser Poindexter subpoena ex-President Reagan personal papers use case against Iran Contra charges ruling Washington U.S. Central American country Nicaragua Sao Paulo Brazil city least 20 people them Ladislav Adamec ending visit Austria Czechoslovakia premier he Prague Communist government Kremlin leader GORBACHEV Soviet Union Soviets Moscow 90 devaluation ruble dollar private transactions action move called House passed chamber THE HOUSE veto legislation renewing support Medicaid abortions poor women are victims rape incest Bush bill easing abortion funding roll call voted only approved antiviral drug AZT sent $ 2.85 billion emergency package assist recovery from last week California earthquake Hurricane Hugo measure Senate changes pesticide law aimed speeding removal harmful chemicals food supply could be announced early today Party facing what is viewed worst unrest nearly 40 East German Krenz coalition political parties Lebanon largest Christian militia A Lebanese accepted plan includes Syrian troop withdrawal Beirut Arab sponsored proposal 14 year civil war direct Israeli Palestinian talks five point Secretary State Baker secretary state Israel Magazine publishers Conference here they publishing industry New York based Hearst Corp. Time Warner increased This Donald Kummerfeld Publishers America first nine months same period Rupert Murdoch News Corp Health Owen Lipstein founder managing partner sale sell James Autry Meredith group some magazines their Magazines Gerry Ritterman Network Publishing I Soap Opera Digest sold They Lang Communications Thoroughbred Owners Breeders Lexington Ky. trade has held dozen seminars so far owners Kentucky business game Tim Hulings beer packaging plant worker Karnak Nile sleek thoroughbred galloping down home stretch just lose average 395,374 summer racehorse ourselves Katherine Voss veteran trainer Laurel Md. track she another investors code ethics go horse something between minority stake total ownership 50 horses Md Spendthrift Farm prominent farm went public 1983 but hit hard times filed bankruptcy protection prices best yearlings IBM said ago was inviting companies participate semiconductor work Motorola paying nominal fee cover 21 month agreement made announcement dedication research center International Business Machines Armonk N.Y. wavelengths even ultraviolet light X-rays agreed project involving invitation join invited those chips costs problems market dynamic random access memory Staar maker products small incision surgery Surgical Co. Thomas R. Waggoner him John Wolf formerly vice sales marketing all directors Chapter 11 petition attempted latest District Judge Edelstein federal judge Manhattan Joseph Lombardo alleged captain organized crime Chicago defendants settle suit figure Brotherhood Teamsters union sweeping racketeering June 1988 1.6 million members judgment requiring pay 250,000 representing profits gained involvement numerous Mobil along other oil producers refiners reduced force 15 during mid-1980s part industrywide shakeout our preparing scheduled reduction staff reductions cutbacks cuts reducing slash yesterday Yesterday third quarter earnings report restructuring Hacksaw Bonecrusher nosed Ted Kimbrough Chancellor Fernandez Former Patterson N.J. principal Joe Clark Elena Scambio woman assigned run Jersey City school system taken over her concept building tenure most disgraceful institutions schools incompetent principals One South Bronx forces status quo superintendents such no more nonsense elected 540 neophyte boards each community parents mistakes competing interests bureaucrats politicians unions awful Chips then mood Morton Downey Jr. self destructed talk host frequently verbally abused guests Mouth Mort nightly Consumer Channel cable channel partly owned General Electric Broadcasting Co Showdown Richard G. Carter columnist Daily CNBC Michael Eskridge Ninety percent That previous shout fest syndciated MCA produced Quantum Media son Seagram stationed many countries around world McDonough 38 architect planet four star Quilted Giraffe restaurant remodeling Paul Stuart Madison Avenue clothing store renovation Poland biggest clean air challenge yet proposed Warsaw Trade Center Europe ambitious Polish Its mesh spire rely 10 square mile forest Sasha Muniak developer All architects headquarters Environmental Defense Fund offices office fund foot ceilings big operable windows unintended side effect : high indoor pollution Ann Hornaday free lance writer visited lunch meetings trees any too worried environment aerobic architecture Dartmouth College neighborhood Alexander Haagen Vermont Slauson Shopping Watts section Los Angeles deteriorated area nothing wig shops shoe surrounding mall opened late 1981 defensible commercial zone shopping creation jobs 500 permanent sector these 2,500 per job investment cost itself repay 60 money inner deals themselves Suburban delays complications typical entire site security command post camouflaged second retail space 70 common vs. These programs Urban Development Action Grants Economic Administration Community Block Industrial Bonds Enterprise Zone write offs infrastructure grants increment financing Zones combines incentives loans encourage depressed areas requirements hiring unemployed minorities markets Government officials wonder how important development risk non-profit Grand Boulevard Plaza developed Matanky Realty Group Third Ward opposite Taylor Homes Boeing strikers rejected raise offer three 55,000 machinists 300 400 aircraft Walter M. Brady Canadian head Frank Ollari department real estate mortgage finance West electronics concerns AEG AG Siemens previously joint venture power semiconductors European Power Semiconductor Germany Groupe Maurice Lippens Belgium insurer Belgian speculators Societe Generale de Belgique S.A. already owns 18 controlled Cie Financiere Suez acquisitive French financial conglomerate giant holding top managers rumored potential raiders -- Axa Midi des Assurances Paris France syndicate shareholders Brussels holdings Asahi Mutual Life Insurance 5 knows believes inclined reflexively compromise beliefs Mrs. Thatcher practicing read my lips politics before encountered Maggie herself She Commonwealth Manchester Guardian paper separate statement Africa everybody else Malaysia 48 opponent throw off course Blue chip advertisers Estee Lauder conference readers reader Leonard niche-itis increasing number target idosyncratic Niche-itis interest editor Conde Nast Publications Our Vanity Fair take Colgate Palmolive unlovable crotchety crusty FBI heart gold intriguing central Mancuso Agent Nick Loggia man investigator good himself mister You you interesting occasionally surprising plot lines -LRB- NBC Fridays ET -RRB- spinoff worse inspirations shows designate accused womanizing la Tower pol Last young black girl Freedom Marchers tailing Edgar Hoover major flaw Executive Producers Steve Sohmer Jeff Bleckner ⁄ Ken Solarz Bello creators writers crisis boss myself politician yourself your key witness trash words crumpled piece pretty line Get soapbox 4 6 damages relief tab needs penny increase six cent temporary rise scaled back calling aid bulk unspecified amount going regions affected influential legislator Willie speaker Assembly Gov. George Deukmejian Governor governor Gann limit existing sources resources help elsewhere gasoline raising gallon taxes set up 1 size Kirk Chamber Commerce men injured triggered freeway collapse Oakland Cypress structure viaduct claims claim Board Control legislature double Anne Volokh Ms. Her husband sons L.A. U.S.S.R Russia Three turban hat Vladimir computer scientist Movieline began national distribution initial press 100,000 copies publication started out listing guide glossy hats boutique owner chirpy folkish music bouncing through room everyone tea Russian Guests favorite minute segments clips Volokhs theirs Their Boris Krutchensky Laurie Halpern Smith now co-editor editorial respected journalist columns vivid discussions cultural literary reverberations well practical advice glamorize dreary Sovietized meals weekly column cuisine cookbook Macmillan bringing soft blini recipe revised works erudite entertaining Art Cuisine brought Dostoevski Kafka mother obvious available occupation translator spoke fluent English goal organization older prodigy profiled Wall Street Journal when 13 Vesoft idea think impossible Pushkin spirit broke restraints convention places Josephine subject 1925 transformed dancing derriere rear end fanny life familiar earlier remarkable cinematic Bricktop found fame Men problem 12 assorted races Rose author Parallel Lives wonderful study Victorian marriage teaches literature Wesleyan University Phyllis Jazz Cleopatra Doubleday 321 pages 22.50 movie like died Free Bryan Hammond photographs drawings pictures Voting Rights Act 1965 act municipal plans citizens racial gerrymandering cities Birmingham Ala 57 Ala. commission appointed mayor revise Charter Revision Commission sitting Council council leaders whites white critics United States Constitution argument representative belief believe Justice Department borough Brooklyn Yet scandal Liberal Democratic party LDP Shigezo Hayasaka aide kingmaker Kakuei Tanaka independent analyst donations quite mainstream pachinko parlor reported annual income trillion yen almost 92 or vaunted automobile Devotees much keep better selling prepaid cards card conservative donated making Socialist JSP North Korean residents Chosen Soren issue Both organizations both groups Recruit influence peddling plagued Calif. Edward Carlson thieves criminals dig plants dead night crooks know botany Sago Palm things Loran Whitelock associate specializing cycads County Arboretum rustlers exotic gardens Harold iron Garden Grove chain AIW 1960s '60s middle class yuppies Lisa Zeidner novel Limited Partnerships Point Press 256 18.95 Nora Worth Malcolm DeWitt 33 39 respectively still artistic aspirations film youngish characters Rascal Money Contemporary Books 412 17.95 Garber innovative widely manufacturing PegaSys tale wars hostile takeover attempt smaller corporation incompetently managed constitute standing joke threat deal Patrician Scott skilled team worlds none art integrity control finished product ; employment without fun profit flame critic actor always seems brink stardom failing assets debts satire descends realm cliched caricatures quake shocked San Francisco registered 6.9 Richter scale disaster county Bay region bay Bridge main artery into east bridge Area Rapid Transit runs subway trains beneath transbay tube BART doubling daily regular ridership 300,000 onslaught price round trip tariff Meetings canceled Apple Computer counties 1.2 miles waterfront highway known Embarcadero Freeway Building 200 downtown rises buildings heavy damage Martin Leach Lloyd London insurance reserve Financial Bakersfield Supermarket Ortiz 29 entrepreneur entrepreneurs there survey businesses responding firms employees Seventeen More than Yale Prof. Albert Reiss monetary losses couple Small quasi-public large relies Only special districts provide additional patrols funded local strategy police presence Westinghouse Shaw Walker Muskegon Mich. Supreme Court decision Webster Reproductive Services opinion majority upheld Missouri Mo. justices drunk driving trespassing false imprisonment activities especially preamble 1986 restored anti-abortion demonstrators blocked clinic St. Louis protesters persons crimes including protester O'Brien Mark Belz lawyer represented succeed resident Columbia charged client motion argued caught drinking ages Daniel Dodson Jefferson definition human cracks Roe Wade auction house Sotheby gamble dealing transaction auctioned Dorrance days T. Campbell Soup heir collection record single 131 clearance Food Drug approval approve cleared treat fatal disease children acquired immune deficiency syndrome youngest AIDS infection FDA given 1,859 under age Elizabeth Glaser activist contracted virus blood transfusion transmitted co-founder Pediatric Foundation Santa Monica bears symptoms daughter Ariel Burroughs Wellcome Philip Pizzo Cancer Institute Helen Kushnick whose Samuel victim tainted My pediatric Democrat Rep. Weiss patients treatment investigational label accorded drugs believed effective lacking formal dementia gradually steals ability speak walk foreign minister SHEVARDNADZE Shevardnadze superpower Anti-Ballistic Missile treaty 1972 ABM accord radar station Krasnoyarsk Afghanistan BAY Hungary Edwards Air Force Base base lawmakers Sunday Aoun peace assistance Western hostages Egyptian President Mubarak official 49 British Gen. PLO Marina fashionable spandex warm suits policeman town rescue shelter Joan O'Shea acupuncturist response primarily elderly devastation homes People badly damaged Barbara May Reds yellows guys Fitzgerald others unbroken ticking pocket watch retrieved n't allowed inside girlfriend policewoman trying remove car Society Prevention Cruelty Animals SPCA parakeet returning apartment salmon steaks prepared chefs restaurants Organization Petroleum Exporting Countries OPEC member chronic cheaters quota 22 barrels rate Oil Herald Tribune Issam Al Chalabi Iraq Iraqi own 28 Saudi Arabia kingpin kingdom Hisham Nazer unused capacity production ceiling boost output Iranian Libya Emirates cheater token 23,000 current Kuwait violinist performed piano accompanist composer McDuffie outgoing pale blue eyes Southern drawl begun violin studies brawny inch musician Young me Yehudi Menuhin Sinfonia 35 Juilliard School conservatory Miss DeLay Galamian longtime assistant ultimately rival noted pedagogue Dorothy grandmotherly looking teacher legendary instructor Ivan student recital listened note B flat dying cancer Barber latter Violin Concerto beautiful Coplandesque motif way marcato bring Schuman recently recorded Angel EMI Bernstein engaging Serenade Solo Strings Percussion sounds mildly atonal rhapsody solo orchestral accompaniment Beethoven ` Macon Ga Itzhak Perlman Isaac Stern Henrik Schwarzenberger Hungarian refugee taught alone indomitable peasant Poles Solidarity sympathetic Bloc liberated bosses Communists Aug. tame Peasants desperate live Peasant farmers hundreds Radzymin buyers Andrzej Latowski farmer butcher Chojnowski truck next Wheat kerchiefed Country monopolies agriculture capitalistic structures Rural countryside offshoot peasants reformers reformer Ownership Czeslaw Pyszkiewicz Mr ground reservoir Farmers protest lay road tractor mechanized equipment farms pension seeds machines Some machinery Felix Siemienas expensive Prime Minister Tadeusz Mazowiecki Next Spring Niciporuk belongs Sergiusz administrator naczelnik thin gray Anatol Pawlowski break acres used moment 76 arable land Zalubice village tumbledown wife Wieslawa Somali regime Siad Barre dictatorship rule declared scientific socialist post-Barre Somalia armed really gang less troops neighboring Ethiopia Marxist port Berbera runway Issak clan Each events horn Horn continent military right surveillance Gulf Aden Indian Ocean lonely strip concrete handful nationals buttressed imported cold drinks issues Sports Illustrated Mengistu Haile Mariam array airfields door bases Kaplan Surrender Starve Wars Behind Famine Westview Tigrean armies Tigreans capital Addis Ababa successor Dese Jewish Jerusalem Palestine Liberation counterterrorism page list recent terrorist incidents attribute directly Chairman Yasser Arafat listed Fatah Yitzhak Shamir adviser Yigal Carmon Times different documents prove conducting terrorism occupied territories says were discovered Gaza leaked acts linked attributes bombs Molotov cocktails experts Dec. dialogue Sears maintains lacks strong ideological pro-choice advocate usually loquacious Vice vacationing popular guy position stance punitive interview Rolling Stone published March 1980 landmark legalizing Ronald threatened pending welfare answer Fowler Ad Hoc Committee Inc activists restrict rights Many Americans firm Republican strategist pollster Wirthlin constitutional amendment outlawing cases banning except endangerment exceptions ruled Florida legislative session regarded test get toughen restrictions administration diamond Namib Desert desert Atlantic ocean sea 336 gargantuan earthmoving De Beers Consolidated Mines Ltd. kingpins earth diamonds wide swath north Orange River inland inhospitable restricted mining precious jewel box treasure chest mine carats Namibia designation normally reserved operations proclaimed Germans Tony engineer artifical wall 600 yards long thick jutting barrier blocks Caterpillar wielding shovels brushes ignominiously named bedrock sweepers toil wake excavators workers bottom bonus equal value Another rising dunes Oranjemund hole golf sand trap Britain account deficit U.K. improvement dropped mild recession pronounced slowdown downturn 1990 Confederation Industry surveyed Wigglesworth economic committee spring Exchequer Nigel Lawson 1989 inflation economist Kleinwort Benson unified economy Sisulu 77 liberation movement ANC outlawed 1960 considered enemy African Congress modest colleagues whom arrested 1963 sentenced racially divided Pretoria seven once formed intellectual organizational core newly released freed aging graying lions boys wing Umkhonto Sizwe wishers Lusaka unconditional release freeing releases Holy Cross Anglican Church Soweto church rally 30 Mandela leading jailed remains prison F.W. Klerk continued enforcement matters oppose concessions townships anti-government protests violence Youth comrades Zambia followers bastion traditional values Trinity Repertory Theater Rep Bogart initially studied directed cites Peter Stein Giorgio Strehler Ariane Mnouchkine influences acclaimed creator deconstructed dramatic collages tear sacred texts Rodgers Hammerstein director Hers season playwright Maxim Gorky Brechtian rigor naturalistic drama helpless play text Summerfolk ennui voiced languorous folk sporting beige linen rumpled cotton boaters fishing poles aplenty decorative nod tradition cast actors individual populating vast multilevel looks parody Wright designed Victoria Petrovich Talk Helpern ever musical equipped spanking 60,000 sound recording studio classics Somewhere Over Rainbow song points view Ed Shea Orson Janice Duclos Marni Rice newcomer Jonathan Fried Zamislov paralegal neighbor Becca Lish Bertolt Brecht Immunex molecule may hold disruptions ranging organ transplant rejection allergies asthma IL receptor surface cells protein interleukin hormone receptors tested Receptech offering agents international environmental attended Lower Silesia Levin EPA regulatory reform participants degraded degradation nitrogen oxide monitoring sites dust soot emissions gross GNP implications shift approaches Market means participant make decisive fiscal spending bills friend vetoed health education 231 191 vote resurgent advocates underlying stopgap disasters Majority Whip Alan Cranston standoff debate GOP Silvio Conte Mass. ranking Appropriations Minority Leader Michel Illinois gubernatorial candidate Florio Dornan Calif fellow Roman Catholics Oregon liberal Les AuCoin 67 providing construction funds incorporating reaching provisions affecting FHA provision Federal Housing 156.7 departments Labor Education Human Delmed makes sells dialysis solution treating kidney diseases Fresenius USA parent holder negotiating acquire controlling pharmaceutical concern Medical Care negotiations Ehrlich buy stock cash beneficial 80 fully diluted combination combine distributor"
mylongstring = unicode(mylongstring, 'utf-8')
doc = nlp(mylongstring)
for token in doc:
    if not is_custom_stopword(token.tag_):
       print token

Famous
Teddy
Z
CBS
Inc.
had
hoped
emerge
few
bright
spots
otherwise
lackluster
prime
time
schedule
centers
mailroom
clerk
turned
agent
Hollywood
talent
agency
half
hour
situation
comedy
seen
Mondays
p.m
Eastern
Pacific
program
series
show
network
coveted
p.m.
slot
lead
character
beginning
premiere
premiered
Greek
family
Craig
Nelson
story
consultant
Mr.
viewers
have
not
been
Murphy
Brown
television
news
magazine
starring
Candice
Bergen
estimated
%
fall
imports
accompanied
growing
tendency
Japanese
manufacturers
build
vehicles
overseas
slip
exports
Japan
Automobile
Manufacturers
Association
association
Domestic
demand
spokesman
new
tax
source
general
resentment
taxpayers
consumption
Robert
J.
Penn
president
chief
executive
officer
years
old
reorganized
company
steelmaker
UNR
William
S.
Leavitt
chairman
father
David
board
nation
alliance
NATO
ministers
concluding
day
meeting
southern
Portugal
defense
court
order
allowing
former
National
Security
Adviser
Poindexter
subpoena
President
Reag

collection
record
single
clearance
Food
Drug
approval
approve
cleared
treat
fatal
disease
children
acquired
immune
deficiency
syndrome
youngest
AIDS
infection
FDA
given
age
Elizabeth
Glaser
activist
contracted
virus
blood
transfusion
transmitted
co
founder
Pediatric
Foundation
Santa
Monica
bears
symptoms
daughter
Ariel
Burroughs
Wellcome
Philip
Pizzo
Cancer
Institute
Helen
Kushnick
Samuel
victim
tainted
pediatric
Democrat
Rep.
Weiss
patients
treatment
investigational
label
accorded
drugs
believed
effective
lacking
formal
dementia
gradually
steals
ability
speak
walk
foreign
minister
SHEVARDNADZE
Shevardnadze
superpower
Anti
Ballistic
Missile
treaty
ABM
accord
radar
station
Krasnoyarsk
Afghanistan
BAY
Hungary
Edwards
Air
Force
Base
base
lawmakers
Sunday
Aoun
peace
assistance
Western
hostages
Egyptian
President
Mubarak
official
British
Gen.
PLO
Marina
fashionable
spandex
warm
suits
policeman
town
rescue
shelter
Joan
O'Shea
acupuncturist
response
primarily
elderly
devastation
homes
People
